In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip -uq "/content/drive/My Drive/dataset.zip" -d "/content/drive/My Drive/dataset"

In [6]:
train = pd.read_csv('/content/drive/My Drive/dataset/dataset/train.csv')
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [7]:
train['target'].value_counts()


mohiniyattam     50
odissi           49
bharatanatyam    47
kathakali        47
kuchipudi        46
sattriya         45
kathak           44
manipuri         36
Name: target, dtype: int64

In [8]:
test = pd.read_csv('/content/drive/My Drive/dataset/dataset/test.csv')


In [9]:

Class_map={'manipuri':0,'kathak':1,'sattriya':2,'kuchipudi':3,'kathakali':4,'bharatanatyam':5,'odissi':6,'mohiniyattam':7}
inverse_map={0:'manipuri',1:'kathak',2:'sattriya',3:'kuchipudi',4:'kathakali',5:'bharatanatyam',6:'odissi',7:'mohiniyattam'}
train['target']=train['target'].map(Class_map)

In [10]:
train_img=[]
train_label=[]
j=0
path='/content/drive/My Drive/dataset/dataset/train'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['target'][j])
    j=j+1

100%|██████████| 364/364 [02:41<00:00,  2.26it/s]


In [11]:
test_img=[]
path='/content/drive/My Drive/dataset/dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 156/156 [01:13<00:00,  2.13it/s]


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagenerator = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.10,  
        height_shift_range=0.10,  
        horizontal_flip=True,  
        vertical_flip=False) 


datagenerator.fit(train_img)

In [13]:

train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(364, 224, 224, 3)
(156, 224, 224, 3)
(364,)


In [14]:
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [15]:

from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.layers import *
from keras.models import Sequential
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape = (224, 224, 3),pooling='avg')
vgg19.trainable = False

model = Sequential([
  vgg19, 
  Dense(1024, activation='relu'),
  Dropout(0.4),
  Dense(256, activation='relu'),
  Dense(8, activation='softmax'),
])

80142336/80134624 [==============================] - 7s 0us/step


In [16]:

from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

vgg19.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(datagenerator.flow(train_img, to_categorical(train_label,8), batch_size=64),
                    epochs=30,callbacks=callbacks)

Epoch 1/30
6/6 [==============================] - 10s 2s/step - loss: 4.8824 - accuracy: 0.2527
Epoch 2/30
6/6 [==============================] - 4s 691ms/step - loss: 1.8686 - accuracy: 0.4835
Epoch 3/30
6/6 [==============================] - 4s 683ms/step - loss: 1.3020 - accuracy: 0.5989
Epoch 4/30
6/6 [==============================] - 4s 684ms/step - loss: 0.9784 - accuracy: 0.6429
Epoch 5/30
6/6 [==============================] - 4s 684ms/step - loss: 0.8592 - accuracy: 0.7225
Epoch 6/30
6/6 [==============================] - 4s 681ms/step - loss: 0.6459 - accuracy: 0.7390
Epoch 7/30
6/6 [==============================] - 4s 685ms/step - loss: 0.5322 - accuracy: 0.7885
Epoch 8/30
6/6 [==============================] - 4s 697ms/step - loss: 0.3795 - accuracy: 0.8571
Epoch 9/30
6/6 [==============================] - 4s 687ms/step - loss: 0.3572 - accuracy: 0.8681
Epoch 10/30
6/6 [==============================] - 4s 677ms/step - loss: 0.2714 - accuracy: 0.9066
Epoch 11/30
6/6 [====

In [17]:

labels = model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.to_csv('result.csv', index=False)